Requirements:
- [Get Started](../01-Day_1/02-Get_started.ipynb)
- [Data Management](../01-Day_1/04-Data_management.ipynb)
- [Atomic Operations](../03-Day_3/02-Atomic_operations.ipynb)

# Deep copy

Complex data structures, including struct and classes in C or derived datatypes with pointers and allocatable in Fortran, are frequent. Ways to managed them include:
- using CUDA unified memory with the compilation flag `-gpu:managed`, but the cost of memory allocation will be higher and it will apply to all allocatable variables
- flatten the derived datatypes by using temporary variables and then perform data transfers on the temporary variables
- using deep copy.

Two ways are possible to manage deep copy:

- bottom-up deep copy with an implicit attach behavior
- top-down deep copy that will be shown in [this notebook.](../03-Day_3/04-Deep_copy_attach_detach.ipynb)

The current notebook describes the top-down deep copy. To implement it we should copy to the device the base structure first and then the children structures. 
For each children transfer, the compiler's implementation will check if the pointers to the children (they are transferred with the parent structure) are present. If they are, an implicit attach behavior is performed and the parents on the device will point toward the children that are newly put on the device. 

Please note that it is not mandatory to transfer all the children structure, only the ones that calculations on the device require.

## Syntax

```C
typedef class{
    float *vx, *vy, *vz;
}velocity;

...
velocity U;
U.vx = (float*) malloc(sizeX*sizeof(float));;
U.vy = (float*) malloc(sizeY*sizeof(float));
U.vz = (float*) malloc(sizeZ*sizeof(float));
...

#pragma acc enter data copy(U)
#pragma acc enter data copy(U.vx[0:sizeX], U.vy[0:sizeY], U.vz[0:sizeZ])

// A humonguous calculation
```

## Example

In this example, we determine the radial distribution function (RDF) for an ensemble of particles that is read from a file. The position of the particles can be use as a demonstration on the implementation of the deep copy. 
You can run this example at the end of the next exercise to check the structure of the box at the end of the simulation.

In [ ]:
%%idrrun -a
//  examples/C/Deep_copy_example.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

typedef struct
{
    double* s;
    double* c;
} Array;

Array* allocate_array(size_t size);

Array* allocate_array(size_t size)
{
    Array* arr = (Array*) malloc(sizeof(Array));
    arr->s = (double*) malloc(size*sizeof(double));
    arr->c = (double*) malloc(size*sizeof(double));
    return arr;
}

int main(void)
{
    int size=1e5;
    double sum[size];

    Array* vec;
    vec = allocate_array(size);

    #pragma acc data create(vec, vec->s[:size], vec->c[:size]) copyout(sum)
    {
    #pragma acc parallel
    {
        #pragma acc loop
        for (int i=0; i<size;++i)
        {
            vec->s[i] = sin(i*M_PI/size)*sin(i*M_PI/size);
            vec->c[i] = cos(i*M_PI/size)*cos(i*M_PI/size);
        }    
    }
    #pragma acc parallel
    {
        #pragma acc loop
        for (int i=0; i<size ; ++i)
            sum[i] = vec->s[i] + vec->c[size - i - 1]; 
    }
    }// end of structured data region
    printf("sum[42] = %f\n", sum[42]);
}


## Exercise

In this exercise, we determine the radial distribution function (RDF) for an ensemble of particles that is read from a file. The position of the particles can be use as a demonstration on the implementation of the deep copy.
You can run this example at the end of the next exercise to check the structure of the box at the end of the simulation.

First you need to copy some files:

In [ ]:
%%bash
cp ../../examples/dyn.xyz .

You need to pass `--cliopts "0.5 15.5"` to idrrun.

In [ ]:
%%idrrun -a
//  examples/C/Deep_copy_exercise.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
# include <math.h>
#include <float.h>

typedef struct
{
    size_t size;
    double* data;
} Array;

typedef struct
{
    Array* x;
    Array* y;
    Array* z;
} Coordinates;

typedef struct
{
    size_t natoms;    // number of atoms
    double lx;        // box length in each dimension
    double ly;         
    double lz;         
} Config;

Array* allocate_array(size_t size);
void free_array(Array* arr);
Coordinates* read_coords(char* filepath, Config *config);
void free_coords(Coordinates* coords);

Array* allocate_array(size_t size)
{
    Array* arr = (Array*) malloc(sizeof(Array));
    arr->size = size;
    arr->data = (double*) malloc(size*sizeof(double));
    return arr;
}

Coordinates* allocate_coords(size_t size)
{
    Coordinates* coords = (Coordinates*) malloc(sizeof(Coordinates));
    coords->x  = allocate_array(size);
    coords->y  = allocate_array(size);
    coords->z  = allocate_array(size);
    return coords;
 }

void free_array(Array* arr)
{
    free(arr->data);
    free(arr);
}

void free_coords(Coordinates* coords)
{
    free_array(coords->x);
    free_array(coords->y);
    free_array(coords->z);
    free(coords);
}

Coordinates* read_coords(char* filepath, Config* conf)
{
    FILE* fptr = fopen(filepath, "r");
    char* line = NULL;
    size_t len = 0;
    size_t i = 0;
    char n[20], x[20], y[20], z[20], vx[20], vy[20], vz[20];
    
    if (fptr == NULL)
        exit(EXIT_FAILURE);

    getline(&line, &len, fptr);
    sscanf(line, "%s", n);
    conf->natoms = atoi(n);
    getline(&line, &len, fptr);
    sscanf(line, "%s", n);
    conf->lx = atof(n);
    conf->ly = atof(n);
    conf->lz = atof(n);
    printf("Number of atoms in %s file: %d\n", filepath, conf->natoms);
    
    Coordinates* coords = allocate_coords(conf->natoms);
    
    while (i < conf->natoms)
    {
        getline(&line, &len, fptr);
        sscanf(line, "%s %s %s %s %s %s %s", n, x, y, z, vx, vy, vz);
        coords->x->data[i] = atof(x);
        coords->y->data[i] = atof(y);
        coords->z->data[i] = atof(z);
	++i;
    }
    fclose(fptr);
    return coords;
}

int main(int argc, char** argv)
{
    double deltaR, rCutOff;
    FILE* fPtr;
    char* input;
    double xij, yij, zij, rij;
    int d;
    
    if (argc < 3 || argc > 4) 
    {
        fprintf(stderr, "%s", "ERROR: Wrong number of parameters.\n");
        fprintf(stderr, "%s", "ERROR: The program requires at least two parameters:\n");
        fprintf(stderr, "%s", "ERROR: deltaR, the length of each bin, and \n");
        fprintf(stderr, "%s", "ERROR: rCutoff, the total length (rcut < box_length/2).\n");
        fprintf(stderr, "%s", "ERROR: Usage example: ./rdf 0.5 15.5 [input]\n");
        exit(EXIT_FAILURE);
    }
    else
    {
        deltaR = atof(argv[1]);
        rCutOff = atof(argv[2]);
        if (argc == 4)
            input = argv[3];
        else
            input = "./dyn.xyz";
    }
    
    int maxbin = rCutOff/deltaR + 1;
    int* hist = (int*) malloc(maxbin*sizeof(int));
    double* gr = (double*) malloc(maxbin*sizeof(double));
    
    for (size_t i=0; i<maxbin; ++i)
        hist[i] = 0;
    
    Config* conf = (Config*) malloc(sizeof(Config));
    Coordinates* coords = read_coords(input, conf);  

    #pragma acc parallel loop present(conf,coords,coords->x,coords->y,coords->z)\
                              present(coords->x->data[:conf->natoms],coords->y->data[:conf->natoms],coords->z->data[:conf->natoms])
    for (int j = 0; j < conf->natoms; ++j)
        #pragma acc loop private(xij,yij,zij,rij,d)
        for (int i = 0; i < conf->natoms; ++i)
            if (i != j)		
            {
	        xij = coords->x->data[j]-coords->x->data[i];
                yij = coords->y->data[j]-coords->y->data[i];
                zij = coords->z->data[j]-coords->z->data[i];
                xij -= floor(xij/conf->lx + 0.5) *conf->lx;
                yij -= floor(yij/conf->ly + 0.5) *conf->ly;
                zij -= floor(zij/conf->lz + 0.5) *conf->lz;
		rij = xij*xij + yij*yij + zij*zij;
                d = (int) (sqrt(rij)/deltaR);
                if (d < maxbin)
                    #pragma acc atomic update
                    ++hist[d];
        }
    
    double rho = ((double) conf->natoms) / ((double)(conf->lx * conf->ly * conf->lz)) * 4.0 / 3.0 * acos(-1.0);
    #pragma acc parallel loop present (hist[:maxbin],gr[:maxbin])
    for (int i = 0; i < maxbin; ++i)
    {
        double nideal  = rho * ( pow((i+1)*deltaR,3) - pow(i*deltaR,3) );
        gr[i] = ((double) hist[i]) / (nideal*conf->natoms);
    }
    #pragma acc update self(gr[:maxbin], hist[:maxbin])
         
    fPtr = fopen("RDF","w");
    for (int i = 0; i < maxbin; ++i)
      fprintf(fPtr,"%lf %lf\n", i*deltaR, gr[i]);
    fclose(fPtr);
    
    free(hist);
    free(gr);
   
    free(conf);
    free_coords(coords);
    return 0;
}

In [ ]:
%%bash
cat $PWD/RDF

## Solution

In [ ]:
%%idrrun -a
//  examples/C/Deep_copy_solution.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <float.h>

typedef struct
{
    size_t size;
    double* data;
} Array;

typedef struct
{
    Array* x;
    Array* y;
    Array* z;
} Coordinates;

typedef struct
{
    size_t natoms;    // number of atoms
    double lx;        // box length in each dimension
    double ly;         
    double lz;         
} Config;

Array* allocate_array(size_t size);
void free_array(Array* arr);
Coordinates* read_coords(char* filepath, Config *config);
void free_coords(Coordinates* coords);

Array* allocate_array(size_t size)
{
    Array* arr = (Array*) malloc(sizeof(Array));
    arr->size = size;
    arr->data = (double*) malloc(size*sizeof(double));
    #pragma acc enter data create(arr, arr->data[:size]) copyin(arr->size)
    return arr;
}

Coordinates* allocate_coords(size_t size)
{
    Coordinates* coords = (Coordinates*) malloc(sizeof(Coordinates));
    #pragma acc enter data create(coords)
    coords->x  = allocate_array(size);
    coords->y  = allocate_array(size);
    coords->z  = allocate_array(size);
    return coords;
 }

void free_array(Array* arr)
{
    free(arr->data);
    free(arr);
    #pragma acc exit data delete(arr->data,arr)
}

void free_coords(Coordinates* coords)
{
    free_array(coords->x);
    free_array(coords->y);
    free_array(coords->z);
    #pragma acc exit data delete(coords)
    free(coords);
}

Coordinates* read_coords(char* filepath, Config* conf)
{
    FILE* fptr = fopen(filepath, "r");
    char* line = NULL;
    size_t len = 0;
    size_t i = 0;
    char n[20], x[20], y[20], z[20], vx[20], vy[20], vz[20];
    
    if (fptr == NULL)
        exit(EXIT_FAILURE);

    getline(&line, &len, fptr);
    sscanf(line, "%s", n);
    conf->natoms = atoi(n);
    getline(&line, &len, fptr);
    sscanf(line, "%s", n);
    conf->lx = atof(n);
    conf->ly = atof(n);
    conf->lz = atof(n);
    #pragma acc enter data copyin(conf)
    printf("Number of atoms in %s file: %d\n", filepath, conf->natoms);
    
    Coordinates* coords = allocate_coords(conf->natoms);
    
    while (i < conf->natoms)
    {
        getline(&line, &len, fptr);
        sscanf(line, "%s %s %s %s %s %s %s", n, x, y, z, vx, vy, vz);
        coords->x->data[i] = atof(x);
        coords->y->data[i] = atof(y);
        coords->z->data[i] = atof(z);
	++i;
    }
    #pragma acc update device(coords->x->data[:conf->natoms],coords->y->data[:conf->natoms],coords->z->data[:conf->natoms])
    fclose(fptr);
    return coords;
}

int main(int argc, char** argv)
{
    double deltaR, rCutOff;
    FILE* fPtr;
    char* input;
    double xij, yij, zij, rij;
    int d;
    
    if (argc < 3 || argc > 4) 
    {
        fprintf(stderr, "%s", "ERROR: Wrong number of parameters.\n");
        fprintf(stderr, "%s", "ERROR: The program requires at least two parameters:\n");
        fprintf(stderr, "%s", "ERROR: deltaR, the length of each bin, and \n");
        fprintf(stderr, "%s", "ERROR: rCutoff, the total length (rcut < box_length/2).\n");
        fprintf(stderr, "%s", "ERROR: Usage example: ./rdf 0.5 15.5 [input]\n");
        exit(EXIT_FAILURE);
    }
    else
    {
        deltaR = atof(argv[1]);
        rCutOff = atof(argv[2]);
        if (argc == 4)
            input = argv[3];
        else
            input = "./dyn.xyz";
    }
    
    int maxbin = rCutOff/deltaR + 1;
    int* hist = (int*) malloc(maxbin*sizeof(int));
    double* gr = (double*) malloc(maxbin*sizeof(double));
    #pragma acc enter data create(hist[:maxbin],gr[:maxbin])
    
    #pragma acc parallel loop present(hist[:maxbin])
    for (size_t i=0; i<maxbin; ++i)
        hist[i] = 0;
    
    Config* conf = (Config*) malloc(sizeof(Config));
    Coordinates* coords = read_coords(input, conf);  

    #pragma acc parallel loop present(conf,coords,coords->x,coords->y,coords->z)\
                              present(coords->x->data[:conf->natoms],coords->y->data[:conf->natoms],coords->z->data[:conf->natoms])
    for (int j = 0; j < conf->natoms; ++j)
        #pragma acc loop private(xij,yij,zij,rij,d)
        for (int i = 0; i < conf->natoms; ++i)
            if (i != j)		
            {
	        xij = coords->x->data[j]-coords->x->data[i];
                yij = coords->y->data[j]-coords->y->data[i];
                zij = coords->z->data[j]-coords->z->data[i];
                xij -= floor(xij/conf->lx + 0.5) *conf->lx;
                yij -= floor(yij/conf->ly + 0.5) *conf->ly;
                zij -= floor(zij/conf->lz + 0.5) *conf->lz;
		rij = xij*xij + yij*yij + zij*zij;
                d = (int) (sqrt(rij)/deltaR);
                if (d < maxbin)
                    #pragma acc atomic update
                    ++hist[d];
        }
    
    double rho = ((double) conf->natoms) / ((double)(conf->lx * conf->ly * conf->lz)) * 4.0 / 3.0 * acos(-1.0);
    #pragma acc parallel loop present (hist[:maxbin],gr[:maxbin])
    for (int i = 0; i < maxbin; ++i)
    {
        double nideal  = rho * ( pow((i+1)*deltaR,3) - pow(i*deltaR,3) );
        gr[i] = ((double) hist[i]) / (nideal*conf->natoms);
    }
    #pragma acc update self(gr[:maxbin])
         
    fPtr = fopen("RDF","w");
    for (int i = 0; i < maxbin; ++i)
      fprintf(fPtr,"%lf %lf\n", i*deltaR, gr[i]);
    fclose(fPtr);
    #pragma acc exit data delete(hist,gr)
    free(hist);
    free(gr);
    #pragma acc exit data delete(conf)
    free(conf);
    free_coords(coords);
    return 0;
}

In [ ]:
%%bash
cat $PWD/RDF